# scratch work

In [8]:
import jax

jax.config.update("jax_enable_x64", True)

from astropy.time import Time

from jorbit.ephemeris import Ephemeris

In [12]:
eph = Ephemeris(ssos="default planets")
eph.processor.state(Time("2020-01-01").tdb.jd)[0].shape

(11, 3)

In [13]:
eph = Ephemeris(ssos="default solar system")
eph.processor.state(Time("2020-01-01").tdb.jd)[0].shape

(27, 3)

In [ ]:
    # "ceres": 2000001,
    # "pallas": 2000002,
    # "juno": 2000003,
    # "vesta": 2000004,
    # "iris": 2000007,
    # "hygiea": 2000010,
    # "eunomia": 2000015,
    # "psyche": 2000016,
    # "euphrosyne": 2000031,
    # "europa": 2000052,
    # "cybele": 2000065,
    # "sylvia": 2000087,
    # "thisbe": 2000088,
    # "camilla": 2000107,
    # "davida": 2000511,
    # "interamnia": 2000704,

# PERTURBER_PACKED_DESIGNATIONS = [
#     "00001",
#     "00002",
#     "00003",
#     "00004",
#     "00007",
#     "00010",
#     "00015",
#     "00016",
#     "00031",
#     "00052",
#     "00065",
#     "00087",
#     "00088",
#     "00107",
#     "00511",
#     "00704",
#     "D4340"
# ]
